In [1]:
#套件
from __future__ import print_function, division
import os  
import pandas as pd
import requests
import numpy as np
import re
import math
import torchvision
import cv2
import glob
import random
from pathlib import Path
from torchvision import datasets,transforms
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
import time
import argparse
from time import sleep
from tqdm import tqdm, trange
from PIL import Image
from torch.autograd import Variable
import torch.nn.functional as FUN
from scipy import io
import efficientnet_pytorch
import torchvision.transforms as T
import PIL
import pickle
import torchvision.datasets as dsets
from scipy.misc import imsave

import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.nn.utils.rnn as rnn_utils
import random
import torch.nn.functional as F

In [ ]:
#製作label和資料集

class_num = 10
#分類標準 0:大幅降低 1:降低 2:不變 3:上升 4:大幅上升

with open('pickle/cifar_train_tensor.pickle', 'rb') as f:
    train_tensor = pickle.load(f)
with open('pickle/cifar_train_label.pickle', 'rb') as f:
    train_label = pickle.load(f)
with open('pickle/cifar_val_tensor.pickle', 'rb') as f:
    val_tensor = pickle.load(f)
with open('pickle/cifar_val_label.pickle', 'rb') as f:
    val_label = pickle.load(f)

with open('pickle/cifar_train_class_label.pickle', 'rb') as f:
    train_class_label = pickle.load(f)

with open('pickle/cifar_val_class_label.pickle', 'rb') as f:
    val_class_label = pickle.load(f)


for class_ in range(class_num):

    count1 = 0
    count2 = 0
    count3 = 0
    count4 = 0
    count5 = 0

    data_set = []
    for t in range(len(val_class_label)):
        differ = val_label[t][0][class_] - val_tensor[t][0][class_]
        if differ >= 0:
            increase = abs(differ) / abs(val_tensor[t][0][class_])

            if increase > 1 :
                classifi = 0
                count5+=1
            elif 1>increase>0.5:
                classifi = 2
                count4+=1
            elif increase<=0.5:
                classifi = 1
                count3+=1

        else:
            reduce = abs(differ) / abs(val_tensor[t][0][class_])

            if reduce > 1 :
                classifi = 4
                count1+=1
            elif 1>reduce>0.5:
                classifi = 2
                count2+=1
            elif reduce<=0.5:
                classifi = 3
                count3+=1


        data_set.append(torch.tensor(classifi))
    a = torch.stack(data_set)
    print(count1,count2,count3,count4,count5)
    # print(count1,count2,count3,count4,count5)
    with open('pickle/cifar_class2/cifar_val_class'+str(class_)+'_label.pickle', 'wb') as f:
        pickle.dump(a, f)



In [3]:
with open(r'C:/Users/Taka\Desktop/efficientnet/active_learning/pickle/cifar_val_tensor.pickle', 'rb') as f:
    val_tensor = pickle.load(f)
with open('pickle/cifar_val_label.pickle', 'rb') as f:
    val_label = pickle.load(f)

print(val_tensor)
print(val_label[5])


[tensor([[ 2.1730, -1.5907,  0.4092, -0.8049, -0.5444, -2.3229,  1.8753, -0.8896,
          1.3429, -0.1002]], device='cuda:0'), tensor([[ 1.6139, -1.7874, -0.1375, -0.3393,  1.8864, -0.4997,  0.0431,  1.1953,
          0.7183, -1.6014]], device='cuda:0'), tensor([[ 4.1065, -1.2227, -0.0208, -1.3185, -0.1947, -0.4899, -2.1748, -0.5434,
          2.7306, -0.7883]], device='cuda:0'), tensor([[ 1.4720, -0.9121,  0.4920, -0.4858,  0.8101, -0.0751, -0.9019,  1.4165,
         -1.1807, -0.9603]], device='cuda:0'), tensor([[ 3.3342, -0.7024,  1.2701, -0.8577, -0.3216, -0.1001, -0.2924, -0.5943,
         -1.4189, -0.7571]], device='cuda:0'), tensor([[ 1.1227, -0.7242,  0.4708,  1.4338, -1.0282, -0.5647, -0.7778, -0.3321,
          0.4742, -0.0732]], device='cuda:0'), tensor([[ 2.9171, -0.8508, -0.2787, -0.9963,  0.4446, -1.5001, -0.6289, -0.2845,
          1.5369, -0.2849]], device='cuda:0'), tensor([[ 1.9330,  0.3626,  1.1107, -0.7893, -1.9115, -1.1304, -2.2351, -0.6269,
          1.3841,  1.9

In [13]:
#製作label和資料集 ( 只取最大的值 )

#分類標準 0:上升 1:不變 3:下降

with open('pickle/cifar_train_tensor.pickle', 'rb') as f:
    train_tensor = pickle.load(f)
with open('pickle/cifar_train_label.pickle', 'rb') as f:
    train_label = pickle.load(f)
with open('pickle/cifar_val_tensor.pickle', 'rb') as f:
    val_tensor = pickle.load(f)
with open('pickle/cifar_val_label.pickle', 'rb') as f:
    val_label = pickle.load(f)

path = 'pickle\cifar_fcn'
class_num = 10
for class_ in range(class_num):

    t_label = []
    t_data = []
    v_label = []
    v_data = []
    count0 = 0
    count1 = 0
    count2 = 0
    count3 = 0
    count4 = 0
    # os.makedirs(path+'/'+'class'+str(class_))

    for t in range(len(train_tensor)):
        value , index = torch.sort(train_tensor[t].squeeze(),descending = True)
        for t2 in range(int(len(index)/10)):

            if int(index[t2]) == class_:
                

                tensor = torch.empty(1,9)
                for t3 in range(9):
                    tensor[0][t3] = (train_tensor[t][0][index[t2]] - train_tensor[t][0][index[t2+t3+1]]) / train_tensor[t][0][index[t2]]
                t_data.append(tensor)


                differ = train_label[t][0][class_] - train_tensor[t][0][class_]
                if differ >= 0:
                    increase = abs(differ) / abs(train_tensor[t][0][class_])

                    if increase > 1 :
                        classifi = 0
                        count0+=1
                    # elif 1.5>=increase>1:
                    #     classifi = 1
                    #     count1+=1
                    # elif 1>=increase>0.5:
                    #     classifi = 2
                    #     count2+=1
                    elif increase<=1:
                        classifi = 1
                        count1+=1

                else:
                    reduce = abs(differ) / abs(train_tensor[t][0][class_])

                    if reduce > 0.5 :
                        classifi = 2
                        count2+=1
                    # elif 1.5>reduce>0.3:
                    #     classifi = 3
                    #     count3+=1
                    elif reduce<=0.5:
                        classifi = 1
                        count1+=1

                t_label.append(torch.tensor(classifi))
    
    da = torch.stack(t_data)
    la = torch.stack(t_label)
    print(len(t_data),'---',len(t_label))
    print(count0,'--',count1,'--',count2,'--',count3,'--',count4)
    with open(path+'/'+'class'+str(class_)+'/cifar_class'+str(class_)+'_label.pickle', 'wb') as f:
        pickle.dump(la, f)
    with open(path+'/'+'class'+str(class_)+'/cifar_class'+str(class_)+'_tensor_distance.pickle', 'wb') as f:
        pickle.dump(da, f)


    for t in range(len(val_tensor)):
        value , index = torch.sort(val_tensor[t].squeeze(),descending = True)
        for t2 in range(int(len(index)/10)):

            if int(index[t2]) == class_:


                tensor = torch.empty(1,9)
                for t3 in range(9):
                    tensor[0][t3] = (val_tensor[t][0][index[t2]] - val_tensor[t][0][index[t2+t3+1]]) / val_tensor[t][0][index[t2]]
                t_data.append(tensor)
                v_data.append(tensor)



                differ = val_label[t][0][class_] - val_tensor[t][0][class_]
                if differ >= 0:
                    increase = abs(differ) / abs(val_tensor[t][0][class_])

                    if increase > 1 :
                        classifi = 0
                        count0+=1
                    # elif 1.5>=increase>1:
                    #     classifi = 1
                    #     count1+=1
                    # elif 1>=increase>0.5:
                    #     classifi = 2
                    #     count2+=1
                    elif increase<=1:
                        classifi = 1
                        count1+=1

                else:
                    reduce = abs(differ) / abs(val_tensor[t][0][class_])

                    if reduce > 0.5 :
                        classifi = 2
                        count2+=1
                    # elif 1.5>reduce>0.3:
                    #     classifi = 3
                    #     count3+=1
                    elif reduce<=0.5:
                        classifi = 1
                        count1+=1

                v_label.append(torch.tensor(classifi))
    
    da2 = torch.stack(v_data)
    la2 = torch.stack(v_label)
    print(len(v_data),'---',len(v_label))
    print(count0,'--',count1,'--',count2,'--',count3,'--',count4)
    with open(path+'/'+'class'+str(class_)+'/cifar_val_class'+str(class_)+'_label.pickle', 'wb') as f:
        pickle.dump(la2, f)
    with open(path+'/'+'class'+str(class_)+'/cifar_val_class'+str(class_)+'_tensor_distance.pickle', 'wb') as f:
        pickle.dump(da2, f)



173 --- 173
78 -- 87 -- 8 -- 0 -- 0
73 --- 73
108 -- 122 -- 16 -- 0 -- 0
170 --- 170
46 -- 118 -- 6 -- 0 -- 0
71 --- 71
64 -- 167 -- 10 -- 0 -- 0
153 --- 153
65 -- 68 -- 20 -- 0 -- 0
57 --- 57
97 -- 86 -- 27 -- 0 -- 0
126 --- 126
71 -- 35 -- 20 -- 0 -- 0
61 --- 61
99 -- 61 -- 27 -- 0 -- 0
191 --- 191
60 -- 107 -- 24 -- 0 -- 0
70 --- 70
87 -- 141 -- 33 -- 0 -- 0
181 --- 181
85 -- 76 -- 20 -- 0 -- 0
75 --- 75
127 -- 104 -- 25 -- 0 -- 0
239 --- 239
65 -- 146 -- 28 -- 0 -- 0
102 --- 102
102 -- 200 -- 39 -- 0 -- 0
141 --- 141
57 -- 77 -- 7 -- 0 -- 0
73 --- 73
90 -- 113 -- 11 -- 0 -- 0
195 --- 195
54 -- 124 -- 17 -- 0 -- 0
82 --- 82
76 -- 173 -- 28 -- 0 -- 0
181 --- 181
83 -- 87 -- 11 -- 0 -- 0
86 --- 86
124 -- 129 -- 14 -- 0 -- 0


In [3]:
#分類網路 function

class Residual(nn.Module):  
    def __init__(self, in_channels, out_channels):
        super(Residual, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=1)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=2, stride=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.bn2 = nn.BatchNorm1d(out_channels)
        
        self.conv4res = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=2, stride=1)
        self.AvgPool1d=nn.AdaptiveAvgPool1d(12)
        self.bn4res = nn.BatchNorm1d(out_channels)

    def forward(self, X):
        
        Y1 = self.conv4res(X)
        Y1 = self.bn4res(Y1)

        Y2 = self.bn1(self.conv1(X))

        Y2 = F.relu(Y2)

        Y2 = self.bn2(self.conv2(Y2))

        Y2 = F.relu(Y2)

        return Y1 + Y2

def resnet_block(in_channels, out_channels, num_residuals):

    blk = []
    for i in range(num_residuals):
        blk.append(Residual(in_channels, out_channels))

    return nn.Sequential(*blk)

class LSTM_FCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super(LSTM_FCN, self).__init__()
        # LSTM
        self.conv4lstm = nn.Conv1d(48, 48, kernel_size=1, stride=1)
        self.rnn = torch.nn.LSTM(input_dim, hidden_dim, num_layers=layers, batch_first=True)

        # 1D conv
        self.conv1 = nn.Conv1d(1, 256, kernel_size=1, stride=1)
        self.res_block=resnet_block(in_channels=256,out_channels=256,num_residuals=3)
        self.conv2 = nn.Conv1d(256, 16, kernel_size=1, stride=1)
        self.AvgPool1d=nn.AdaptiveAvgPool1d(48) # length

        # concat softmax
        self.fc1 = nn.Linear(768, 256, bias=True)
        self.dropou1=nn.Dropout(0.7)
        self.fc2 = nn.Linear(256, 64, bias=False)
        self.dropou2=nn.Dropout(0.7)
        self.fc3 = nn.Linear(64, output_dim, bias=False)
        self.softmax=nn.Softmax(dim=1)

    def init_model(self):
        for m in self.modules():
            if isinstance(m, (torch.nn.Linear, torch.nn.Conv1d)):
                torch.nn.init.xavier_uniform_(m.weight)
        print("init success !!")

    def forward(self, x):
        
        
        
        # x=torch.unsqueeze(x,0)
        # 1D cnn
        cnn_out=self.conv1(x)
        cnn_out=self.res_block(cnn_out)
        cnn_out=self.conv2(cnn_out)
        cnn_out=self.AvgPool1d(cnn_out)            

        y=torch.flatten(cnn_out,start_dim=1)
   
        y = self.fc1(y) 
        y=self.dropou1(y)
  
        y = self.fc2(y) 
        y=self.dropou2(y)
        y = self.fc3(y) 

        y= self.softmax(y)
 
        return y

In [4]:
device = 'cuda'
MyResModel = LSTM_FCN(input_dim=1, hidden_dim=16, output_dim=3, layers=1).to(device)
MyResModel.init_model()

init success !!


In [5]:
device = 'cuda'
criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(LSTM.parameters(),lr=0.001)
optimizer = torch.optim.SGD(MyResModel.parameters(), lr=0.0001, momentum=0.9)

In [22]:
#train分類模型

Epochs=500
max_test_acc=0.0
class_number = 10
train_loss_list=[]
train_acc_list=[]
test_loss_list=[]
test_acc_list=[]
batch_size = 32
device = 'cuda'
model_path = 'linear_weight/cifar'

'''匯入所有label'''
train_label_ = []
for t in range(class_number):
    with open('pickle\cifar_fcn/class'+str(t)+'/cifar_class'+str(t)+'_label.pickle', 'rb') as f:
        train_label_.append(pickle.load(f))


val_label_ = []
for t in range(class_number):
    with open('pickle\cifar_fcn/class'+str(t)+'/cifar_val_class'+str(t)+'_label.pickle', 'rb') as f:
        val_label_.append(pickle.load(f))


'''匯入所有confidence'''
train_tensor_ = []
for t in range(class_number):
    with open('pickle\cifar_fcn/class'+str(t)+'/cifar_class'+str(t)+'_tensor.pickle', 'rb') as f:
        train_tensor_.append(pickle.load(f))


val_tensor_ = []
for t in range(class_number):
    with open('pickle\cifar_fcn/class'+str(t)+'/cifar_val_class'+str(t)+'_tensor.pickle', 'rb') as f:
        val_tensor_.append(pickle.load(f))


#訓練各類模型
for class_n in range(class_number):
    print('class '+str(class_n)+' model')
    # 按batch size包裝training set資料
    register_1 = []
    register_2 = []
    train_tensor_list = []
    train_label_list = []
    count = 0
    best_acc = 0
    for t in range(len(train_tensor_[class_n])):
        count+=1
        # numpy = FUN.softmax(Variable(train_tensor_[class_n][t])).data.cpu().numpy()#把input從confidence轉為機率


        #對input做預分配權重( 置信度高的權重愈重 )
        # value , index = torch.sort(train_tensor_[class_n][t].squeeze(),descending = True)
        # for t2 in range(len(index)):
        #     if t2<2:
        #         pass
        #     else:
        #         numpy[0][index[t2]] = numpy[0][index[t2]] * (0.8**t2)
        #         # train_tensor_[class_n][t] = train_tensor_[class_n][t]* (0.8**t2)


        # tensor = torch.tensor(numpy)
        embedding = nn.Embedding(11,20,padding_idx=0)

        value , index = torch.sort(train_tensor_[class_n][t].squeeze(),descending = True)
        input = torch.tensor([1,2,3,4,5,6,7,8,9,10])

        for tt in range(len(index)):
            input[index[tt]] = tt 

        a = embedding(input)
        a = a.to(device)
        b = torch.mm(train_tensor_[class_n][t],a)
        tensor = b
        # tensor = train_tensor_[class_n][t]
        register_1.append(tensor)
        register_2.append(train_label_[class_n][t])
        if count % batch_size ==0:
            a = torch.stack(register_1)
            b = torch.stack(register_2)
            train_tensor_list.append(a)
            train_label_list.append(b)
            register_1 = []
            register_2 = []

    #依bach size打包validation set
    register_1 = []
    register_2 = []
    val_tensor_list = []
    val_label_list = []
    count = 0
    for t in range(len(val_tensor_[class_n])):
        count+=1
        # numpy = FUN.softmax(Variable(val_tensor_[class_n][t])).data.cpu().numpy()#把input從confidence轉為機率
        # tensor = torch.tensor(numpy)

        embedding = nn.Embedding(11,20,padding_idx=0)
        value , index = torch.sort(val_tensor_[class_n][t].squeeze(),descending = True)
        input = torch.tensor([1,2,3,4,5,6,7,8,9,10]).to(torch.int64)
        for tt in range(len(index)):
            input[index[tt]] = tt 
        a = embedding(input)
        a = a.to(device)
        b = torch.mm(val_tensor_[class_n][t],a)
        tensor = b

        # tensor = val_tensor_[class_n][t]
        register_1.append(tensor)
        register_2.append(val_label_[class_n][t])
        if count % batch_size ==0:
            a = torch.stack(register_1)
            b = torch.stack(register_2)
            val_tensor_list.append(a)
            val_label_list.append(b)
            register_1 = []
            register_2 = []


    #開始練模型
    for epoch in range(Epochs):
        train_loss=0.0
        train_acc=0.0
        test_loss=0.0
        test_acc=0.0
        MyResModel.train()
        count = 0
        train_loss_count = 0
        train_acc_count = 0
        for data_ in train_tensor_list:
            optimizer.zero_grad()
            pred = MyResModel(data_.to(device))

            max_,class_ = torch.max(pred.data,1)
            train_correct = (class_==train_label_list[count].to(device)).sum()
            train_acc = train_correct / batch_size
            train_acc_count = train_acc_count + train_acc.item()

            loss = criterion(pred.to(device),train_label_list[count].to(device))
            train_loss_count = train_loss_count+loss.item()

            count+=1
            loss.backward(retain_graph=True)
            # loss.backward()
            optimizer.step()
        MyResModel.eval()
        a = (epoch+1) / 100
        if (epoch+1) % 10 ==0:
            print('epoch : '+ str(int(a*100)) +' train_loss : '+str(train_loss_count/count)+' train_Acc : '+str(train_acc_count/count))

        #算驗證集loss和acc
        if (epoch+1) % 100 ==0:
            count = 0
            loss_count = 0
            acc_count = 0
            for data_ in val_tensor_list:
                pred = MyResModel(data_.to(device))
                max_,class_ = torch.max(pred.data,1)
                correct = (class_==val_label_list[count].to(device)).sum()
                acc = correct / batch_size
                acc_count = acc_count + acc.item()
                loss = criterion(pred.to(device),val_label_list[count].to(device))
                loss_count = loss_count+loss.item()
                count+=1
            if acc_count/count > best_acc:
                best_acc = acc_count/count
                torch.save(MyResModel.state_dict(), model_path+'/'+'model'+str(class_n)+'.pth')
            a = (epoch+1) / 100
            print()
            print('epoch : '+ str(a*100) +' loss : '+str(loss_count/count)+' Acc : '+str(acc_count/count))
            print() 

class 0 model
epoch : 10 train_loss : 0.9830449938774108 train_Acc : 0.5625
epoch : 20 train_loss : 0.8643725991249085 train_Acc : 0.68125
epoch : 30 train_loss : 0.839653480052948 train_Acc : 0.70625
epoch : 40 train_loss : 0.8100564002990722 train_Acc : 0.7375
epoch : 50 train_loss : 0.7583438634872437 train_Acc : 0.7875
epoch : 60 train_loss : 0.7727297782897949 train_Acc : 0.775
epoch : 70 train_loss : 0.7553063273429871 train_Acc : 0.7875
epoch : 80 train_loss : 0.7244921565055847 train_Acc : 0.825
epoch : 90 train_loss : 0.7329418540000916 train_Acc : 0.8125
epoch : 100 train_loss : 0.7133373498916626 train_Acc : 0.83125

epoch : 100.0 loss : 1.1131592392921448 Acc : 0.421875

epoch : 110 train_loss : 0.6890656590461731 train_Acc : 0.8625
epoch : 120 train_loss : 0.7011347532272338 train_Acc : 0.85
epoch : 130 train_loss : 0.6703206300735474 train_Acc : 0.875
epoch : 140 train_loss : 0.691665780544281 train_Acc : 0.85625
epoch : 150 train_loss : 0.6708118319511414 train_Acc : 0.8

KeyboardInterrupt: 

In [15]:
#用train出來的分類模型對各個sample的預測結果進行分類

with open('pickle\cifar_data/alldata_result.pickle', 'rb') as f:
    result = pickle.load(f)

with open('pickle\cifar_data/alldata_class_label.pickle', 'rb') as f:
    label = pickle.load(f)

with open('pickle\cifar_data/alldata_img_path.pickle', 'rb') as f:
    path = pickle.load(f)


weight_dir = 'linear_weight/cifar'
weight_path = os.listdir(weight_dir)
model_file = []

for t in range(len(weight_path)):
    path = weight_dir + '/' + weight_path[t]
    model_file.append(path)


# device = 'cuda'
# MyResModel = LSTM_FCN(input_dim=1, hidden_dim=16, output_dim=5, layers=1).to(device)
# MyResModel.init_model()

count0 = 0
count1 = 0
count2 = 0
count3 = 0
count4 = 0

for t in range(len(model_file)):
    MyResModel.load_state_dict(torch.load(model_file[t]))
    MyResModel = MyResModel.to(device)
    for t2 in range(len(result)):
        numpy = FUN.softmax(Variable(result[t2])).data.cpu().numpy()
        tensor = torch.tensor(numpy)
        tensor = tensor.unsqueeze(0)
        tensor = tensor.to(device)
        output = MyResModel(tensor)
        pre_class = torch.argmax(output)

        if int(pre_class)==0:
            result[t2][0][t] = result[t2][0][t] + result[t2][0][t]*0.5
            count0+=1
        if int(pre_class)==1:
            # result[t2][0][t] = result[t2][0][t] - result[t2][0][t]*0.2
            result[t2][0][t] = result[t2][0][t]
            count1+=1
        if int(pre_class)==2:
            result[t2][0][t] = result[t2][0][t]- result[t2][0][t]*0.2
            count2+=1
        # if int(pre_class)==3:
        #     result[t2][0][t] = result[t2][0][t] + result[t2][0][t]*0.2
        #     count3+=1
        # if int(pre_class)==4:
        #     result[t2][0][t] = result[t2][0][t] + result[t2][0][t]*0.5
        #     count4+=1

acc_count = 0
for t in range(len(result)):        
        r = torch.argmax(result[t])
        if int(r) == label[t]:
            acc_count+=1
print('acc : '+str(acc_count/len(result)))
print(count0,count1,count2,count3,count4)
with open('pickle/cifar_data2/alldata_revise_result.pickle', 'wb') as f:
    pickle.dump(result, f)

c:\Users\Taka\anaconda3\envs\taka1\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


acc : 0.80612
187965 298807 13228 0 0


In [15]:
#用train出的模型修改本來的confidence( 只修改前兩大的 )

with open('pickle\cifar_data/alldata_result.pickle', 'rb') as f:
    result = pickle.load(f)

with open('pickle\cifar_data/alldata_class_label.pickle', 'rb') as f:
    label = pickle.load(f)

with open('pickle\cifar_data/alldata_img_path.pickle', 'rb') as f:
    path = pickle.load(f)


weight_dir = 'linear_weight/cifar'
weight_path = os.listdir(weight_dir)
model_file = []
count0 = 0
count1 = 0
count2 = 0
count3 = 0
count4 = 0

for t in range(len(weight_path)):
    path = weight_dir + '/' + weight_path[t]
    model_file.append(path)

for t in range(len(result)):     
    value , index = torch.sort(result[t].squeeze(),descending = True)
    for t2 in range(int(len(index)/10)):
        MyResModel.load_state_dict(torch.load(model_file[int(index[t2])]))
        MyResModel = MyResModel.to(device)
        numpy = FUN.softmax(Variable(result[t])).data.cpu().numpy()
        tensor = torch.tensor(numpy)
        tensor = tensor.unsqueeze(0)
        tensor = tensor.to(device)
        output = MyResModel(tensor)
        pre_class = torch.argmax(output)

        if int(pre_class)==0:
            result[t][0][int(index[t2])] = result[t][0][int(index[t2])] + result[t][0][int(index[t2])]*0.5
            count0+=1
        if int(pre_class)==1:
            result[t][0][int(index[t2])] = result[t][0][int(index[t2])]
            count1+=1
        if int(pre_class)==2:
            result[t][0][int(index[t2])] = result[t][0][int(index[t2])]- result[t][0][int(index[t2])]*0.2
            count2+=1
acc_count = 0
for t in range(len(result)):        
        r = torch.argmax(result[t])
        if int(r) == label[t]:
            acc_count+=1
print('acc : '+str(acc_count/len(result)))
print(count0,count1,count2,count3,count4)
with open('pickle/cifar_data2/alldata_revise_just_max.pickle', 'wb') as f:
    pickle.dump(result, f)       

c:\Users\Taka\anaconda3\envs\taka1\lib\site-packages\ipykernel_launcher.py:31: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


acc : 0.8214
44964 4776 260 0 0


In [23]:
#改完confidence後 算enstropy挑 x% data

# with open('pickle\cifar_data/alldata_revise_result.pickle', 'rb') as f:
#     result = pickle.load(f)

with open('pickle\cifar_data/alldata_class_label.pickle', 'rb') as f:
    label = pickle.load(f)

with open('pickle\cifar_data/alldata_img_path.pickle', 'rb') as f:
    path = pickle.load(f)

with open('pickle\cifar_data2/alldata_revise_just_max3.pickle', 'rb') as f: 
    result = pickle.load(f)

'''算entropy'''
def entropy(input):
    all = 0
    for t in range(len(input)):
        if input[t]>=0:
            en = -(input[t]*math.log(input[t],2))
            all = all+en
    return all

img_save_dir = 'cifar_10/cifar10_50percent_version2' 
img_dir = 'cifar_10\cifar10'
all_en = []
for t in range(len(result)):
    odds = FUN.softmax(Variable(result[t]).cpu()).data.numpy()
    odds = odds.reshape([10])
    data_entropy = entropy(odds)
    all_en.append(data_entropy)
sort = sorted(range(len(all_en)) , reverse = True,key = lambda k : all_en[k])
img_dir_list=glob.glob(img_dir+os.sep+"*")#獲取每個類别所在的路徑（一個類别對應一個文件夾）
for class_dir in img_dir_list:
    class_name=class_dir.split(os.sep)[-1] #獲取當前類别
    save_train=img_save_dir+os.sep+os.sep+class_name
    os.makedirs(save_train,exist_ok=True)#建立對應的文件夾

for t in range(int(len(sort)*0.5)):
    p = re.sub("\,","",str(path [sort[t]]))
    p = re.sub("\(","",p)
    p = re.sub("\)","",p)
    p = re.sub("\'","",p)
    split = p.split('\\')

    img = cv2.imread(p)
    cv2.imwrite(img_save_dir+'/'+ split[4]+'/'+split[6], img)

c:\Users\Taka\anaconda3\envs\taka1\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [8]:
#用改完的confidence算enstropy挑 x% data(各類數量平均)

with open('pickle\cifar_data/alldata_class_label.pickle', 'rb') as f:
    label = pickle.load(f)

with open('pickle\cifar_data/alldata_img_path.pickle', 'rb') as f:
    path = pickle.load(f)

with open('pickle\cifar_data2/alldata_revise_just_max.pickle', 'rb') as f: 
    result = pickle.load(f)

class_num = 10

'''算entropy'''
def entropy(input):
    all = 0
    for t in range(len(input)):
        if input[t]>=0:
            en = -(input[t]*math.log(input[t],2))
            all = all+en
    return all

img_save_dir = 'cifar_10/register' 
img_dir = 'cifar_10\cifar10'
all_en = []
for t in range(len(result)):
    odds = FUN.softmax(Variable(result[t]).cpu()).data.numpy()
    odds = odds.reshape([10])
    data_entropy = entropy(odds)
    all_en.append(data_entropy)
sort = sorted(range(len(all_en)) , reverse = True,key = lambda k : all_en[k])

img_dir_list=glob.glob(img_dir+os.sep+"*")#獲取每個類别所在的路徑（一個類别對應一個文件夾）
for class_dir in img_dir_list:
    class_name=class_dir.split(os.sep)[-1] #獲取當前類别
    save_train=img_save_dir+os.sep+os.sep+class_name
    os.makedirs(save_train,exist_ok=True)#建立對應的文件夾

con = [0 for t in range(class_num)]
for t in range(int(len(sort))):
    p = re.sub("\,","",str(path [sort[t]]))
    p = re.sub("\(","",p)
    p = re.sub("\)","",p)
    p = re.sub("\'","",p)
    split = p.split('\\')

    if con[int(split[4])] <= (int(len(sort))/class_num) *0.5:

        con[int(split[4])] = con[int(split[4])]+1
        img = cv2.imread(p)
        cv2.imwrite(img_save_dir+'/'+ split[4]+'/'+split[6], img)

c:\Users\Taka\anaconda3\envs\taka1\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
